<a href="https://colab.research.google.com/github/juanmed/dw_a/blob/main/DLL_package_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 30kB/s 
     |████████████████████████████████| 3.0MB 44.2MB/s 
     |████████████████████████████████| 460kB 51.9MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2449, done.
remote: Counting objects: 100% (2449/2449), done.
remote: Compressing objects: 100% (2049/2049), done.
remote: Total 2449 (delta 588), reused 1358 (delta 373), pack-reused 0
Receiving objects: 100% (2449/2449), 30.85 MiB | 37.70 MiB/s, done.
Resolving deltas: 100% (588/588), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1624302 sha256=19dd8434583613292c6aeaea4908db235d654ce5af7e9d269bedc32c08339162
  Stored in directory: /tmp/pip-ephem-wheel-cache-4t6f9ptd/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp37-none-any.whl size=43735 sha256=bb59e72ed9e2ea93e7672a6a83c534b4227a152d5d5175e972772129712a7ace
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=c151c25f4f678e45237663dcc1de7b654c6ad98f686ec987032286cd36d1598b
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=74eb51750cada2c37a4e6267644a493e4aec88c552b19a287a53

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


In [7]:
import pathlib as pl

training_path = "./workspace/training_demo"
annotations_path = "./annotations"

if not os.path.exists(training_path+"/images/train"):
  pl.Path(training_path+"/images/train").mkdir(parents=True, exist_ok=True)
if not os.path.exists(training_path+"/test"):
  pl.Path(training_path+"/images/test").mkdir(parents=True, exist_ok=True)
if not os.path.exists(annotations_path):
  pl.Path(annotations_path).mkdir(parents=True, exist_ok=True)

In [8]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=04f19ca4824c071b765ad981b4010fc650a0050282b686a8234520259ba6a71a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [9]:
import wget
dataset_partition_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/dataset_partition.py"
dataset_partition_file = wget.download(dataset_partition_url)

In [19]:
# Partition the dataset
%%bash
python dataset_partition.py -x -i "./workspace/training_demo/images" -r 0.1

In [20]:
gen_tf_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/generate_tfrecord.py"
wget.download(gen_tf_url)

'generate_tfrecord (1).py'

In [34]:
import shutil
labelmap_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/label_map.pbtxt"
labelmap_file = wget.download(labelmap_url)

anno_path = "./workspace/training_demo/annotations"
if not os.path.exists(anno_path):
  pl.Path(anno_path).mkdir(parents=True, exist_ok=True)

shutil.move(labelmap_file, anno_path+"/"+labelmap_file)

'./workspace/training_demo/annotations/label_map.pbtxt'

In [36]:
# Generate .record files
%%bash
python generate_tfrecord.py -x ./workspace/training_demo/images/train -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/train.record
python generate_tfrecord.py -x ./workspace/training_demo/images/test -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/test.record

Successfully created the TFRecord file: ./workspace/training_demo/annotations/train.record
Successfully created the TFRecord file: ./workspace/training_demo/annotations/test.record


In [26]:
# Download pretrained EfficienteNet D5 1280x1280 from
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


In [27]:

import sys
effnet_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d5_coco17_tpu-32.tar.gz"
filename = wget.download(effnet_url)

In [28]:
import tarfile

pretrained_path = "./workspace/training_demo/pre-trained-models"
if not os.path.exists(pretrained_path):
  pl.Path(pretrained_path).mkdir(parents=True, exist_ok=True)

tar = tarfile.open(filename) 
tar.extractall(path=pretrained_path) # untar file into same directory
tar.close()

In [32]:
# Configure model
models_path = "./workspace/training_demo/models/efficientdet_d5_coco17_tpu-32"
if not os.path.exists(models_path):
  pl.Path(models_path).mkdir(parents=True, exist_ok=True)

config_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/pipeline.config"
config_file = wget.download(config_url)

shutil.move(config_file, models_path + "/" + config_file)
shutil.copy("./models/research/object_detection/model_main_tf2.py","./workspace/training_demo/model_main_tf2.py")

'./workspace/training_demo/models/efficientdet_d5_coco17_tpu-32/pipeline.config'

In [37]:
# Start training
%%bash
cd workspace/training_demo/
python model_main_tf2.py --model_dir=models/efficientdet_d5_coco17_tpu-32 --pipeline_config_path=models/efficientdet_d5_coco17_tpu-32/pipeline.config

2021-03-05 01:04:21.695589: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-05 01:04:23.928319: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-05 01:04:23.929236: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-05 01:04:23.958954: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-05 01:04:23.959597: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-05 01:04:23.959653: I tensorflow/stream_executor/platform/default/dso_loade